## **Este archivo se corre (y está) en la instancia de 4X**

In [2]:
!cat /proc/meminfo

MemTotal:       65951648 kB
MemFree:        37176592 kB
MemAvailable:   37860364 kB
Buffers:          150332 kB
Cached:          1077208 kB
SwapCached:            0 kB
Active:         27720968 kB
Inactive:         705148 kB
Active(anon):   27198600 kB
Inactive(anon):       60 kB
Active(file):     522368 kB
Inactive(file):   705088 kB
Unevictable:           0 kB
Mlocked:               0 kB
SwapTotal:             0 kB
SwapFree:              0 kB
Dirty:                68 kB
Writeback:             0 kB
AnonPages:      27198824 kB
Mapped:           159040 kB
Shmem:                72 kB
Slab:             135840 kB
SReclaimable:      95412 kB
SUnreclaim:        40428 kB
KernelStack:        8352 kB
PageTables:        59068 kB
NFS_Unstable:          0 kB
Bounce:                0 kB
WritebackTmp:          0 kB
CommitLimit:    32975824 kB
Committed_AS:   30042804 kB
VmallocTotal:   34359738367 kB
VmallocUsed:           0 kB
VmallocChunk:          0 kB
AnonHugePages:         0 kB
ShmemHugePages:  

In [2]:
import pandas as pd
import numpy as np
import io
import gc
import boto3
import re

# ruta de trabajo en s3
bucket = 'boi-banregio'

# s3 resource
s3_bucket_resource = boto3.resource('s3').Bucket(bucket)
seed = 0

In [3]:
# Aquí colocamos la ruta de nuestro directorio, el nombre de nuestro archivo y si tiene o no cabecera
prefix = 'datalake/data/InteligenciaRiesgos/M&M/MCV'

data_file_name = "JAT_MCV_VAR_VARIABLES_SELECCION/JAT_MCV_VAR_VARIABLES_SELECCION3.csv"

label_file_name = "JAT_MCV_VAR_VARIABLES_SELECCION/JAT_MCV_VAR_VARIABLES_SELECCION3_LABELS.csv"
# configuración de datos crudos
data_file_key = "{}/DATASETS_3/{}".format(prefix, data_file_name)
label_file_key = "{}/DATASETS_3/{}".format(prefix, label_file_name)
# data_file_key = "datalake/data/InteligenciaRiesgos/M&M/MCV/DATASETS_3/JAT_MCV_VAR_VARIABLES_SELECCION/JAT_MCV_VAR_VARIABLES_SELECCION3.csv"

In [4]:
label_obj = s3_bucket_resource.Object(label_file_key).get()
label_df = pd.read_csv(io.BytesIO(label_obj['Body'].read()), header=None)

In [20]:
label_df

,0,1,2,3,4,5,6,7,8,9,...,3894,3895,3896,3897,3898,3899,3900,3901,3902,3903
0,LABEL,BMI,REVOLVENTE,CUR_BAL,MAX_BAL_03M,MIN_BAL_03M,AVG_PND_MENOS_BAL_03M,AVG_PND_MAS_BAL_03M,AVG_BAL_03M,MAX_BAL_06M,...,PCT_UN_CRE_CS_OP_ACC,PCT_UN_CRE_LSG_CL_ACC,PCT_UN_CRE_LSG_OP_ACC,PCT_UN_CRE_OP_ACC,PCT_UN_CRE_OTHER_CL_ACC,PCT_UN_CRE_OTHER_OP_ACC,PCT_UN_CRE_PQ_CL_ACC,PCT_UN_CRE_PQ_OP_ACC,PCT_UN_CRE_R_CL_ACC,PCT_UN_CRE_R_OP_ACC


In [ ]:
rfc_key_features_file_name = "DATASETS_3/JAT_MCV_VAR_VARIABLES_SELECCION/PARTITION/EXTRA_FEATURES/MCV_VAR_RFC_FECHA_LLAVE.csv"
rfc_key_features_file_key = "{}/{}".format(prefix, rfc_key_features_file_name)
rfc_key_features_obj = s3_bucket_resource.Object(rfc_key_features_file_key).get()
rfc_key_features_data = pd.read_csv(io.BytesIO(rfc_key_features_obj['Body'].read()))

In [5]:
parts = 10
np.random.seed(seed)
rand_split = np.random.rand(len(label_df.columns)-2)  # drop LABEL, BMI labels


bool_partition =[]
for j in range(0, parts):
    appnd = (rand_split >= j/parts) & (rand_split < (j+1)/parts)
    print(appnd)
    bool_partition.append(appnd)
len(bool_partition)

[False False False ... False False False]
[False False False ... False False  True]
[False False False ...  True False False]
[False False False ... False False False]
[False False False ... False False False]
[ True False False ... False  True False]
[False False  True ... False False False]
[False  True False ... False False False]
[False False False ... False False False]
[False False False ... False False False]


10

In [6]:
data_obj = s3_bucket_resource.Object(data_file_key).get()
df = pd.read_csv(io.BytesIO(data_obj['Body'].read()))

In [9]:
df.columns

Index(['LABEL', 'BMI', 'REVOLVENTE', 'CUR_BAL', 'MAX_BAL_03M', 'MIN_BAL_03M',
       'AVG_PND_MENOS_BAL_03M', 'AVG_PND_MAS_BAL_03M', 'AVG_BAL_03M',
       'MAX_BAL_06M',
       ...
       'PCT_UN_CRE_CS_OP_ACC', 'PCT_UN_CRE_LSG_CL_ACC',
       'PCT_UN_CRE_LSG_OP_ACC', 'PCT_UN_CRE_OP_ACC', 'PCT_UN_CRE_OTHER_CL_ACC',
       'PCT_UN_CRE_OTHER_OP_ACC', 'PCT_UN_CRE_PQ_CL_ACC',
       'PCT_UN_CRE_PQ_OP_ACC', 'PCT_UN_CRE_R_CL_ACC', 'PCT_UN_CRE_R_OP_ACC'],
      dtype='object', length=3904)

In [11]:
data_targets = df.loc[:,['LABEL', 'BMI']]
print('targets loaded')
target_file_name = "JAT_MCV_VAR_VARIABLES_SELECCION/PARTITION/TARGETS.csv"
target_obj_key = "{}/DATASETS_3/{}".format(prefix, target_file_name)

f_str = io.StringIO()
data_targets.to_csv(f_str, index=False)
s3_bucket_resource.Object(target_obj_key).put(Body=f_str.getvalue())


targets loaded


{'ResponseMetadata': {'RequestId': '6422B8AD1C5772ED',
  'HostId': 'sFg33XkBs4l0HDtMZG/Ksh+li/EeuvcY71RqH6e0onFwH+GcTtN1tTI9kcCtT5nML+b1+UURo48=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'sFg33XkBs4l0HDtMZG/Ksh+li/EeuvcY71RqH6e0onFwH+GcTtN1tTI9kcCtT5nML+b1+UURo48=',
   'x-amz-request-id': '6422B8AD1C5772ED',
   'date': 'Tue, 04 Sep 2018 01:55:15 GMT',
   'etag': '"3610872f5b797e47bd66816220f891ea"',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'ETag': '"3610872f5b797e47bd66816220f891ea"'}

In [13]:
data_features = df.drop(
    ['LABEL', 'BMI'],
    axis=1
)

In [16]:
data_features.columns

Index(['REVOLVENTE', 'CUR_BAL', 'MAX_BAL_03M', 'MIN_BAL_03M',
       'AVG_PND_MENOS_BAL_03M', 'AVG_PND_MAS_BAL_03M', 'AVG_BAL_03M',
       'MAX_BAL_06M', 'MIN_BAL_06M', 'AVG_PND_MENOS_BAL_06M',
       ...
       'PCT_UN_CRE_CS_OP_ACC', 'PCT_UN_CRE_LSG_CL_ACC',
       'PCT_UN_CRE_LSG_OP_ACC', 'PCT_UN_CRE_OP_ACC', 'PCT_UN_CRE_OTHER_CL_ACC',
       'PCT_UN_CRE_OTHER_OP_ACC', 'PCT_UN_CRE_PQ_CL_ACC',
       'PCT_UN_CRE_PQ_OP_ACC', 'PCT_UN_CRE_R_CL_ACC', 'PCT_UN_CRE_R_OP_ACC'],
      dtype='object', length=3902)

In [22]:

for k in range(0, parts):
    data_part_file_name = "JAT_MCV_VAR_VARIABLES_SELECCION/PARTITION/DATA_PART{}.csv".format(k)
    data_part_obj_key =  "{}/DATASETS_3/{}".format(prefix, data_part_file_name)
    data_part = data_features.loc[:, bool_partition[k]]
    
    f_str = io.StringIO()
    data_part.to_csv(f_str, index=False)
    s3_bucket_resource.Object(data_part_obj_key).put(Body=f_str.getvalue())
    #del f_str
    #gc.collect()
    print('step: ', k)


step:  0
step:  1
step:  2
step:  3
step:  4
step:  5
step:  6
step:  7
step:  8
step:  9
